'mydrop0.2-maxf120-cleannum' の 一番いいアンサンブル で、 my_lstmのsnapshot ensemble
dropemb0.1のモデルをそれぞれ入れ替える
my_lstm_attenを'facal0.1-drop0.15-maxf120-cleannum'に変える
original-cnn1d-glove' を 'facal0.1-drop0.2-maxf120-cleannum'に変える
concatで'my_lstm_atten-glove'モデルを変える
my-lstmは 最後の2epochをensemble

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
## some config values 
embed_size = 300 # how big is each word vector
max_features = 120000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 70 # max number of words in a question to use

**Load packages and data**

In [ ]:
import os
import re
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate, Conv1D, MaxPool1D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.layers import Lambda, Dot
from keras.layers import MaxPool1D, AveragePooling1D
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers

In [ ]:
assert len(K.tensorflow_backend._get_available_gpus()) > 0
K.tensorflow_backend._get_available_gpus()

os.environ['PYTHONHASHSEED'] = '0'
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
import random as rn
rn.seed(7)

In [ ]:

puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]
def clean_text(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, f' {punct} ')
    return x

def split_text(x):
    x = wordninja.split(x)
    return '-'.join(x)

def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

In [ ]:
def load_and_prec():
    train_df = pd.read_csv("../input/train.csv")
    test_df = pd.read_csv("../input/test.csv")
    
    train_df["question_text"] = train_df["question_text"].str.lower()
    test_df["question_text"] = test_df["question_text"].str.lower()
    
    train_df["question_text"] = train_df["question_text"].apply(lambda x: clean_text(x))
    test_df["question_text"] = test_df["question_text"].apply(lambda x: clean_text(x))
    
    train_df["question_text"] = train_df["question_text"].apply(lambda x: clean_numbers(x))
    test_df["question_text"] = test_df["question_text"].apply(lambda x: clean_numbers(x))
    
    print("Train shape : ",train_df.shape)
    print("Test shape : ",test_df.shape)

    ## fill up the missing values
    train_X = train_df["question_text"].fillna("_##_").values
    test_X = test_df["question_text"].fillna("_##_").values

    ## Tokenize the sentences
    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(list(train_X))
    train_X = tokenizer.texts_to_sequences(train_X)
    test_X = tokenizer.texts_to_sequences(test_X)

    ## Pad the sentences 
    train_X = pad_sequences(train_X, maxlen=maxlen)
    test_X = pad_sequences(test_X, maxlen=maxlen)

    ## Get the target values
    train_y = train_df['target'].values
    
    #shuffling the data
    np.random.seed(2018)
    trn_idx = np.random.permutation(len(train_X))

    train_X = train_X[trn_idx]
    train_y = train_y[trn_idx]
    
    return train_X, test_X, train_y, tokenizer.word_index, test_df['qid']

**Load embeddings**

In [ ]:
def load_glove(word_index):
    EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = -0.005838499,0.48782197
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
            
    return embedding_matrix 
    
def load_fasttext(word_index):    
    EMBEDDING_FILE = '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector

    return embedding_matrix

def load_para(word_index):
    EMBEDDING_FILE = '../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = -0.0053247833,0.49346462
    embed_size = all_embs.shape[1]
    print(emb_mean,emb_std,"para")

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    
    return embedding_matrix

In [ ]:
def focal_loss(y_true, y_pred):
    gamma = 0.1
    true_loss = K.mean(y_true * ((1-y_pred)**gamma * K.log(y_pred + K.epsilon())), axis=0)
    false_loss = K.mean((1-y_true) * (y_pred**gamma * K.log(1-y_pred + K.epsilon())), axis=0)
    return - (true_loss + false_loss)

**CNN Model**

In [ ]:
# https://www.kaggle.com/yekenot/2dcnn-textclassifier
def model_cnn_1d(embedding_matrix):
    filter_sizes = [1,2,3,5]
    num_filters = 36

    inp = Input(shape=(maxlen,)) # batch sizeを含まない
    # maxlen: 70
    # max_features: indexの数
    # embed_size: embedの次元 300 先頭できまってる
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp) # (batch, embeddim, input_length)
    x = Reshape((maxlen, embed_size))(x) # batch, input_length, embed_size

    maxpool_pool = []
    for i in range(len(filter_sizes)):
        conv = Conv1D(num_filters, kernel_size=filter_sizes[i],
                                     kernel_initializer='he_normal', activation='elu')(x)
        maxpool_pool.append(MaxPool1D(pool_size=maxlen - filter_sizes[i] + 1)(conv))

    z = Concatenate(axis=1)(maxpool_pool)   
    z = Flatten()(z)
    z = Dropout(0.1)(z)

    outp = Dense(1, activation="sigmoid")(z)

    model = Model(inputs=inp, outputs=outp)
    model.compile(loss=focal_loss, optimizer='adam', metrics=['accuracy'])
    
    return model

**Attention layer**

In [ ]:
# https://www.kaggle.com/suicaokhoailang/lstm-attention-baseline-0-652-lb

class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

**LSTM models**

In [ ]:
def model_lstm_atten_mix(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
    x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)
    x = Attention(maxlen)(x)
    x = Dense(64, activation="elu")(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

def model_lstm_atten_glove(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = Dropout(0.1)(x)
    x = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
    x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)
    x = Attention(maxlen)(x)
    x = Dense(64, activation="relu")(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [ ]:
def model_lstm_du(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
def model_my_lstm(embedding_matrix):
    p_drop = 0.1

    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size*2, weights=[embedding_matrix], trainable=False)(inp)
    x = Dropout(p_drop)(x)

    x = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
    x = Dropout(p_drop)(x)

    x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)
    
    atten_x = Attention(maxlen)(x)
        
    max_x = MaxPool1D(maxlen)(x) # 1, units
    max_x = Flatten()(max_x) # units
    
    ave_x = AveragePooling1D(maxlen)(x) # 1, units
    ave_x = Flatten()(ave_x) # units
    
    x = Concatenate()([atten_x, max_x, ave_x]) # units * 6
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss=focal_loss, optimizer='adam', metrics=['accuracy'])
    
    return model

**Train and predict**

In [ ]:
# https://www.kaggle.com/strideradu/word2vec-and-gensim-go-go-go
def train_pred(model, epochs):
    for e in range(epochs):
        model.fit(train_X, train_y, batch_size=512, epochs=1, verbose=2)
    pred_test_y = model.predict([test_X], batch_size=1024, verbose=0)
    return pred_test_y

def train_pred_snapshot(model, epochs, start_epoch):
    pred_list = []
    for e in range(epochs):
        model.fit(train_X, train_y, batch_size=512, epochs=1, verbose=2)
        if e >= start_epoch:
            pred_test_y = model.predict([test_X], batch_size=1024, verbose=0)
            pred_list.append(pred_test_y)
    
    return pred_list

**Main part: load, train, pred and blend**

In [ ]:
%%time
train_X, test_X, train_y, word_index, test_df = load_and_prec()

In [ ]:
%%time
embedding_matrix_glove = load_glove(word_index)

In [ ]:
%%time
embedding_matrix_para = load_para(word_index)

In [ ]:
embedding_matrix_mix = np.mean([embedding_matrix_glove, embedding_matrix_para], axis = 0)
embedding_matrix_concat = np.concatenate([embedding_matrix_glove, embedding_matrix_para], axis = 1)

In [ ]:
%%time
outputs = []

pred_test_y = train_pred(model_cnn_1d(embedding_matrix_glove), epochs = 2) # GloVe only
outputs.append([pred_test_y, '1d-CNN'])

In [ ]:
%%time
pred_test_y = train_pred(model_lstm_du(embedding_matrix_mix), epochs = 2)
outputs.append([pred_test_y, 'LSTM-DU'])

In [ ]:
%%time
pred_test_y = train_pred(model_lstm_atten_mix(embedding_matrix_mix), epochs = 3)
outputs.append([pred_test_y, '2-LSTM-attention-mix'])

In [ ]:
%%time
pred_test_y = train_pred(model_lstm_atten_glove(embedding_matrix_glove), epochs = 3)
outputs.append([pred_test_y, '2-LSTM-attention-glov'])

In [ ]:
%%time
pred_test_y_list = train_pred_snapshot(model_my_lstm(embedding_matrix_concat), epochs=5, start_epoch=3)
outputs.extend([[pred, 'my_lstm'] for pred in pred_test_y_list])

In [ ]:
%%time
w_list = [0.15573309, 0.15155236, 0.20362984, 0.1206986,  0.14503201, 0.2233541]
w_normed = np.array(w_list) / np.sum(w_list)
pred_test_y = np.sum([outputs[i][0]*w_normed[i] for i in range(len(w_normed))], axis = 0)
pred_test_y = (pred_test_y > 0.3434).astype(int)

In [ ]:
%%time
out_df = pd.DataFrame({"qid":test_df.values})
out_df['prediction'] = pred_test_y
out_df.to_csv("submission.csv", index=False)